## 향후 7일 입도객 예측

#### 패키지 임포트

In [1]:
# Ignore the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# Data manipulation, visualization and useful functions
import argparse
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from datetime import date, timedelta
import missingno as msno

# gcp functions
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

# Keras and tensorflow
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Flatten, Dropout
from keras.layers import SimpleRNN, LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import layers
import keras_tuner

#### 데이터 쿼리 from 빅쿼리

In [2]:
# Set up BigQuery clients
bqclient = bigquery.Client(project='charged-genre-350106')

# Query for base dataset

query_AP = """
    SELECT
      CONCAT(CAST(FORMAT_DATE("%E4Y", CAST(DT as date)) AS string),
             CAST(FORMAT_DATE("%m", CAST(DT as date)) AS string),
             CAST(FORMAT_DATE("%d", CAST(DT as date)) AS string)) AS DT
      , ARRIVE_PPL as ARRIVE_PPL
    FROM `charged-genre-350106.kaflix.AIRLINE_PASSENGER`
    """

query_NS = """
    SELECT *
    FROM `charged-genre-350106.kaflix.NAVER_SEARCH`
    """

query_AT = """
    WITH MAIN AS (
        SELECT *
        FROM kaflix.AIRLINE_TICKET A
        WHERE 1=1
            -- AND A.DEPART = 'GMP'
            -- AND A.ARRIVE = 'CJU'
        AND A.SEARCH_DATE NOT IN ('2022-07-08', '2022-07-09', '2022-07-10',  '2022-07-12')
    )
    SELECT
        A.*
        , B.B
        , B.F
        , B.D
        , B.S
        , B.SD
    FROM(
        SELECT
           CONCAT(CAST(FORMAT_DATE("%E4Y", CAST(LEFT(DEPART_DATE,10) as date)) AS string),
                  CAST(FORMAT_DATE("%m", CAST(LEFT(DEPART_DATE,10) as date)) AS string),
                  CAST(FORMAT_DATE("%d", CAST(LEFT(DEPART_DATE,10) as date)) AS string)) AS DEPART_DT
           , CONCAT(CAST(FORMAT_DATE("%E4Y", CAST(LEFT(SEARCH_DATE, 10) as date)) AS string),
                    CAST(FORMAT_DATE("%m", CAST(LEFT(SEARCH_DATE,10) as date)) AS string),
                    CAST(FORMAT_DATE("%d", CAST(LEFT(SEARCH_DATE,10) as date)) AS string)) AS SEARCH_DT
           , CAST(FORMAT_DATE("%a", CAST(LEFT(DEPART_DATE, 10) as date)) AS string) AS DOW
           , DATE_DIFF(CAST(LEFT(DEPART_DATE,10) AS date), CAST(LEFT(SEARCH_DATE,10) as date), DAY) AS LEAD_TM
        , COUNT(*) AS TICKET
        , MIN(FARE) AS FARE_MIN
        , MAX(FARE) AS FARE_MAX
        , AVG(FARE) AS FARE_AVG
        , STDDEV(FARE) AS FARE_STD
        , MIN(AVAIL_SEAT) AS SEAT_MIN
        , MAX(AVAIL_SEAT) AS SEAT_MAX
        , AVG(AVAIL_SEAT) AS SEAT_AVG
        , STDDEV(AVAIL_SEAT) AS SEAT_STD
        FROM MAIN
        WHERE 1=1
        GROUP BY 
        DEPART_DATE, SEARCH_DATE
    ) A
    LEFT JOIN (
        SELECT *
        FROM(
            SELECT
            CONCAT(CAST(FORMAT_DATE("%E4Y", CAST(LEFT(DEPART_DATE,10) as date)) AS string), CAST(FORMAT_DATE("%m", CAST(LEFT(DEPART_DATE,10) as date)) AS string), CAST(FORMAT_DATE("%d", CAST(LEFT(DEPART_DATE,10) as date)) AS string)) AS DEPART_DT
            , CONCAT(CAST(FORMAT_DATE("%E4Y", CAST(LEFT(SEARCH_DATE, 10) as date)) AS string), CAST(FORMAT_DATE("%m", CAST(LEFT(SEARCH_DATE,10) as date)) AS string), CAST(FORMAT_DATE("%d", CAST(LEFT(SEARCH_DATE,10) as date)) AS string)) AS SEARCH_DT 
                , CLASS_DESC
            FROM MAIN
            WHERE 1=1
        ) A
        PIVOT (
            COUNT(*) 
            FOR CLASS_DESC IN ('비즈니스석' AS B, '일반석' AS F, '할인석' AS D, '특가석' AS S, '단독특가' AS SD)
        )
    ) B ON A.DEPART_DT = B.DEPART_DT AND A.SEARCH_DT = B.SEARCH_DT
    WHERE 1=1
        -- AND A.DEPART_DT < '20220725'
    ORDER BY 
      A.DEPART_DT DESC
      , A.LEAD_TM
    """

query_HD = """
    SELECT
      CONCAT(CAST(FORMAT_DATE("%E4Y", CAST(DT as date)) AS string),
             CAST(FORMAT_DATE("%m", CAST(DT as date)) AS string),
             CAST(FORMAT_DATE("%d", CAST(DT as date)) AS string)) AS DT
      , TOURIST AS TOURIST
      , TEMPERTURE AS TEMPERTURE
      , RAIN AS RAIN 
      , HOLIDAY_NAME AS HOLIDAY_NAME
    FROM `charged-genre-350106.kaflix.TOURIST_WEATHER`
    """

query_RC = """
    SELECT
      PARSE_DATE('%Y%m%d', PURCHASE_DATE) AS PURCHASE_DT
      , PARSE_DATE('%Y%m%d', BEGIN_DATE) AS BEGIN_DT
      , DATE_DIFF (PARSE_DATE('%Y%m%d', BEGIN_DATE),PARSE_DATE('%Y%m%d', PURCHASE_DATE), DAY) AS LEAD_TM
    FROM `charged-genre-350106.kaflix.ERP_MERGING`
    """

query_TW = """
    SELECT *
    FROM `charged-genre-350106.kaflix.TWAY_KAFLIX`
    """

In [3]:
tw = bqclient.query(query_TW).to_dataframe()

In [4]:
tw

,SEARCH_DATE,FLIGHT_NUMBER,DEPART_DATE,DEPART,DEPART_TIME,ARRIVE_TIME,FLIGHT_MODEL,TOTAL_SEAT,GROUP_SOLD,TOTAL_SOLD
0,2022-03-11,TW505,2022-03-11,GMP,2022-03-11 08:15:00,2022-03-11 09:30:00,330,1,0,1
1,2022-03-11,TW507,2022-03-11,GMP,2022-03-11 13:40:00,2022-03-11 14:55:00,330,10,0,2
2,2022-03-14,TW509,2022-03-14,GMP,2022-03-14 10:00:00,2022-03-14 11:15:00,330,8,0,2
3,2022-03-15,TW517,2022-03-15,GMP,2022-03-15 11:00:00,2022-03-15 12:20:00,330,6,0,3
4,2022-01-01,TW701,2022-01-01,GMP,2022-01-01 06:40:00,2022-01-01 07:50:00,737,189,7,169
...,...,...,...,...,...,...,...,...,...,...
424785,2022-03-22,TW9945,2022-03-26,PUS,2022-03-26 09:55:00,2022-03-26 10:55:00,737,189,7,109
424786,2022-03-23,TW9945,2022-03-26,PUS,2022-03-26 09:55:00,2022-03-26 10:55:00,737,189,7,113
424787,2022-03-24,TW9945,2022-03-26,PUS,2022-03-26 09:55:00,2022-03-26 10:55:00,737,189,5,116
424788,2022-03-25,TW9945,2022-03-26,PUS,2022-03-26 09:55:00,2022-03-26 10:55:00,737,189,5,189


#### 데이터 로드 from 빅쿼리

In [5]:
## Data Load
def data_load(AP, AT, HD, RC, TW, start_date, end_date):
    psg = bqclient.query(AP).to_dataframe()
    air = bqclient.query(AT).to_dataframe()
    wth = bqclient.query(HD).to_dataframe()
    rc = bqclient.query(RC).to_dataframe()
    tw = bqclient.query(TW).to_dataframe()
    
    # converting datetime
    psg['DT']=pd.to_datetime(psg['DT'])
    air['DEPART_DT']=pd.to_datetime(air['DEPART_DT'])
    wth['DT']=pd.to_datetime(wth['DT'])
    rc['BEGIN_DT']=pd.to_datetime(rc['BEGIN_DT'])
    tw['SEARCH_DATE']=pd.to_datetime(tw['SEARCH_DATE'])
    tw['DEPART_DATE']=pd.to_datetime(tw['DEPART_DATE'])

    # air ticket 7days lagged to predict next 7days
    air = air.query("8<=LEAD_TM <= 28")
    rc=rc.query('8<=LEAD_TM<=28')

    # Indexing pgs date & left join with other data
    psg.set_index("DT", inplace=True)
    psg = psg.loc[(psg.index >= start_date) & (psg.index<= end_date),:]  # 0513까지 삭제
    psg = psg.groupby("DT")['ARRIVE_PPL'].sum() # 일별 입도객 합계
    psg = pd.DataFrame(psg)                     # 데이터프레임만들고
    
    # column rename to DT
    air.rename(columns={'DEPART_DT':'DT'}, inplace=True)
    rc.rename(columns={'BEGIN_DT':'DT'}, inplace=True)
    #tw.rename(columns={'DEPART_DATE':'DT'}, inplace=True)

    wth = wth[['DT','TEMPERTURE','RAIN','HOLIDAY_NAME']]
    air = air[['DT','LEAD_TM','TICKET','FARE_MIN','FARE_MAX','FARE_AVG', 'FARE_STD', 'SEAT_MIN', 'SEAT_MAX', 'SEAT_AVG', 'SEAT_STD','B','F','D','S','SD']]
    return psg, air, wth, rc, tw

### 렌타카 데이터 전처리

In [6]:
def rentacar_processing(df):
    df=df.groupby(['PURCHASE_DT','DT','LEAD_TM']).size().reset_index()
    df.columns=['purchase_dt','DT', 'lead_time','count']
    df=df.pivot_table(index=['DT'], columns='lead_time', values='count')
    df=df.reset_index()
    return df

### 티웨이 데이터 전처리

In [7]:
def tway_processing(df):
    df1=df[['SEARCH_DATE','DEPART_DATE','FLIGHT_MODEL']]
    df1=df1.groupby(['SEARCH_DATE','DEPART_DATE','FLIGHT_MODEL']).size().reset_index()
    df1=df1.pivot_table(index=['SEARCH_DATE','DEPART_DATE'], columns='FLIGHT_MODEL', aggfunc=['sum']).reset_index()
    df1['lead_time']=df1['DEPART_DATE']-df1['SEARCH_DATE']
    df1['lead_time']=df1['lead_time'].dt.days
    df1.columns=df1.columns.to_flat_index()
    df1.columns=['search_date','depart_date','330','737','lead_time']
    df1=df1.fillna(0)

    
    df2=df[['SEARCH_DATE','DEPART_DATE','GROUP_SOLD','TOTAL_SOLD']]
    df2=df2.groupby(['SEARCH_DATE','DEPART_DATE']).sum().reset_index()
    df2['lead_time']=df2['DEPART_DATE'] - df2['SEARCH_DATE']
    df2['lead_time']=df2['lead_time'].dt.days
    df2.columns=['search_date','depart_date','group_sold','total_sold','lead_time']
    
    df3=pd.merge(df1, df2, how='left', on=['search_date','depart_date'])
    df3=df3.drop('lead_time_y', axis=1)
    df3.columns=['search_date','depart_date','group_sold','total_sold','lead_time','330','737']
    df3=df3.query('8<=lead_time<=28')
    
    df3=df3.pivot_table(index=['depart_date'], columns='lead_time', values=['group_sold','total_sold','330','737'])
    df3.columns=df3.columns.to_flat_index()
    df3=df3.reset_index()
    df3.columns=['DT','330_8','330_9','330_10','330_11','330_12','330_13','330_14',
                      '330_15','330_16','330_17','330_18','330_19','330_20','330_21',
                      '330_22','330_23','330_24','330_25','330_26','330_27','330_28',
                      '737_8','737_9','737_10','737_11','737_12','737_13','737_14',
                      '737_15','737_16','737_17','737_18','737_19','737_20','737_21',
                      '737_22','737_23','737_24','737_25','737_26','737_27','737_28',
                      'group_sold_8','group_sold_9','group_sold_10','group_sold_11','group_sold_12','group_sold_13','group_sold_14',
                      'group_sold_15','group_sold_16','group_sold_17','group_sold_18','group_sold_19','group_sold_20','group_sold_21',
                      'group_sold_22','group_sold_23','group_sold_24','group_sold_25','group_sold_26','group_sold_27','group_sold_28',
                      'total_sold_8','total_sold_9','total_sold_10','total_sold_11','total_sold_12','total_sold_13','total_sold_14',
                      'total_sold_15','total_sold_16','total_sold_17','total_sold_18','total_sold_19','total_sold_20','total_sold_21',
                      'total_sold_22','total_sold_23','total_sold_24','total_sold_25','total_sold_26','total_sold_27','total_sold_28']
    return df3

#### 휴일 데이터 전처리

In [8]:
## Holiday Data Preprocessing
def holiday_data_pre(df):
    wth_h = df[['DT', 'HOLIDAY_NAME']]

    # 1 for all holidays , 0 for others
    wth_h.HOLIDAY_NAME.loc[~wth_h.HOLIDAY_NAME.isnull()] = 1
    wth_h = wth_h.sort_values('DT')
    wth_h.HOLIDAY_NAME.loc[wth_h.HOLIDAY_NAME.isnull()] = 0
    wth_h.HOLIDAY_NAME.unique()

    # [long holidays] and [Holidays Interspersed with Workdays]
    wth_h['dayofweek'] = wth_h.DT.dt.dayofweek

    # 공휴일인데 금요일(4)이면 그주 금요일(4)을 연휴시작으로
    # 공휴일인데 월요일(0)이면 전주 토요일(5)을 연휴시작으로
    wth_h['flong']=np.where((wth_h['HOLIDAY_NAME'] == 1) & (wth_h['dayofweek'] == 4), 1, 0)
    wth_h['mlong']=np.where((wth_h['HOLIDAY_NAME'] == 1) & (wth_h['dayofweek'] == 0), 1, 0)

    #월요일 연휴는 그 전 토요일에 연휴 시작 표시
    wth_h['mlong']= wth_h['mlong'].shift(-2)
    wth_h['mlong'].fillna(method='ffill', inplace = True)

    #금요일 연휴, 월요일 연휴 컬럼 합치기
    wth_h['long_h']=wth_h['flong'] + wth_h['mlong'] 

    #연휴시작 하루 전날 표시
    wth_h['b_long_h']=wth_h['long_h'].shift(-1)
    wth_h['b_long_h'].fillna(method='ffill', inplace = True)

    # 징검다리 로직
    # 휴일인데 목요일(3)이면 그주 목요일(3)을 징검다리 연휴시작
    # 휴일인데 화요일(1)이면 전주 토요일(5)을 징검다리 연휴시작
    wth_h['thinter'] = np.where((wth_h['HOLIDAY_NAME'] == 1) & (wth_h['dayofweek'] == 3), 1, 0)
    wth_h['tuinter'] = np.where((wth_h['HOLIDAY_NAME'] == 1) & (wth_h['dayofweek'] == 1), 1, 0)

    # 화요일 징검다리는 전주 토요일에 연휴 시작 표시
    wth_h['tuinter'] = wth_h['tuinter'].shift(-3)
    wth_h['tuinter'].fillna(method='ffill', inplace = True)
    wth_h['inter_h'] = wth_h['thinter'] + wth_h['tuinter'] 

    # 징검다리 시작 하루 전날 표시
    wth_h['b_inter_h'] = wth_h['inter_h'].shift(-1)
    wth_h['b_inter_h'].fillna(method='ffill', inplace = True)

    # 명절 연휴 시작일 및 시작일 전날 표시 
    #holiday    : 설날/추석 명절 -> 1
    #first_m    : 설날/추석 명절연휴 시작일,단 명절 연휴시작이 (일) 또는 (월)이면 (토)가 시작일 -> 1 표시
    #b_first_m  : 설날/추석 명절연휴 시작일 전날, 단 명절 연휴시작이 (일) 또는 (월)이면 (금)이 시작일 전날 -> 1로 표시

    #first_m    : 설날/추석 명절연휴 시작일,단 명절 연휴시작이 (일) 또는 (월)이면 (토)가 시작일 -> 1 표시
    #2017-01-27, 2017-10-03, 2018-02-15, 2018-09-23, 2019-02-04, 2019-09-12, 2020-01-24, 2020-09-30, 2021-02-11, 2021-09-30, 2022-02-01

    #first_m_nosm   : 설날/추석 명절연휴 시작일,단 명절 연휴시작이 (화) 또는 (수) 또는 (목) 또는 (금) 또는 (토) 이면 1
    wth_h['first_m_nosm']=np.where(((wth_h['DT'] == '2017-01-27') |
                              (wth_h['DT'] == '2017-10-03') |
                              (wth_h['DT'] == '2018-02-15') |
                              (wth_h['DT'] == '2018-09-23') |
                              (wth_h['DT'] == '2019-02-04') |
                              (wth_h['DT'] == '2019-09-12') |
                              (wth_h['DT'] == '2020-01-24') |
                              (wth_h['DT'] == '2020-09-30') |
                              (wth_h['DT'] == '2021-02-11') |
                              (wth_h['DT'] == '2021-09-20') |
                              (wth_h['DT'] == '2022-01-31') |
                              (wth_h['DT'] == '2022-09-09') )
                              & ((wth_h['dayofweek'] != 6) & (wth_h['dayofweek'] !=0)), 1,0)

    #first_m_s   : 설날/추석 명절연휴 시작일,단 명절 연휴시작이 (일)이면 1로 표시하고 shift -1
    wth_h['first_m_s']=np.where(((wth_h['DT'] == '2017-01-27') |
                              (wth_h['DT'] == '2017-10-03') |
                              (wth_h['DT'] == '2018-02-15') |
                              (wth_h['DT'] == '2018-09-23') |
                              (wth_h['DT'] == '2019-02-04') |
                              (wth_h['DT'] == '2019-09-12') |
                              (wth_h['DT'] == '2020-01-24') |
                              (wth_h['DT'] == '2020-09-30') |
                              (wth_h['DT'] == '2021-02-11') |
                              (wth_h['DT'] == '2021-09-20') |
                              (wth_h['DT'] == '2022-01-31') |
                              (wth_h['DT'] == '2022-09-09') )
                              & (wth_h['dayofweek'] == 6), 1,0)
    wth_h['first_m_s'] = wth_h['first_m_s'].shift(-1)
    wth_h['first_m_s'].fillna(method='ffill', inplace = True)

    #first_m_m   : 설날/추석 명절연휴 시작일,단 명절 연휴시작이 (월)이면 1로 표시하고 shift -2
    wth_h['first_m_m']=np.where(((wth_h['DT'] == '2017-01-27') |
                              (wth_h['DT'] == '2017-10-03') |
                              (wth_h['DT'] == '2018-02-15') |
                              (wth_h['DT'] == '2018-09-23') |
                              (wth_h['DT'] == '2019-02-04') |
                              (wth_h['DT'] == '2019-09-12') |
                              (wth_h['DT'] == '2020-01-24') |
                              (wth_h['DT'] == '2020-09-30') |
                              (wth_h['DT'] == '2021-02-11') |
                              (wth_h['DT'] == '2021-09-20') |
                              (wth_h['DT'] == '2022-01-31') |
                              (wth_h['DT'] == '2022-09-09') )
                              & (wth_h['dayofweek'] == 0), 1,0)
    wth_h['first_m_m'] = wth_h['first_m_m'].shift(-2)
    wth_h['first_m_m'].fillna(method='ffill', inplace = True)

    wth_h['first_m'] = wth_h['first_m_nosm'] + wth_h['first_m_s'] + wth_h['first_m_m']

    wth_h['b_first_m'] = wth_h['first_m'].shift(-1)
    wth_h['b_first_m'].fillna(method='ffill', inplace=True)

    wth_h.drop(['flong', 'mlong', 'thinter','tuinter', 'first_m_nosm','first_m_s', 'first_m_m'], axis=1, inplace=True)
    return wth_h

#### 항공티켓 전처리

In [9]:
# LOUIE'S CELL
def air_dataprocessing(air):
    rename = ['DT', 'ticket_8', 'ticket_9', 'ticket_10','ticket_11','ticket_12','ticket_13','ticket_14',
                    'ticket_15','ticket_16', 'ticket_17','ticket_18','ticket_19','ticket_20','ticket_21',
                    'ticket_22', 'ticket_23', 'ticket_24','ticket_25','ticket_26','ticket_27','ticket_28',
                    'fare_min_8','fare_min_9','fare_min_10','fare_min_11','fare_min_12','fare_min_13','fare_min_14',
                    'fare_min_15','fare_min_16','fare_min_17','fare_min_18','fare_min_19','fare_min_20','fare_min_21',
                    'fare_min_22','fare_min_23','fare_min_24','fare_min_25','fare_min_26','fare_min_27','fare_min_28',
                    'fare_max_8','fare_max_9','fare_max_10','fare_max_11','fare_max_12','fare_max_13','fare_max_14',
                    'fare_max_15','fare_max_16','fare_max_17','fare_max_18','fare_max_19','fare_max_20','fare_max_21',
                    'fare_max_22','fare_max_23','fare_max_24','fare_max_25','fare_max_26','fare_max_27','fare_max_28',
                    'fare_avg_8','fare_avg_9','fare_avg_10','fare_avg_11','fare_avg_12','fare_avg_13','fare_avg_14',
                    'fare_avg_15','fare_avg_16','fare_avg_17','fare_avg_18','fare_avg_19','fare_avg_20','fare_avg_21',
                    'fare_avg_22','fare_avg_23','fare_avg_24','fare_avg_25','fare_avg_26','fare_avg_27','fare_avg_28',
                    'fare_std_8','fare_std_9','fare_std_10','fare_std_11','fare_std_12','fare_std_13','fare_std_14',
                    'fare_std_15','fare_std_16','fare_std_17','fare_std_18','fare_std_19','fare_std_20','fare_std_21',
                    'fare_std_22','fare_std_23','fare_std_24','fare_std_25','fare_std_26','fare_std_27','fare_std_28',
                    'seat_min_8','seat_min_9','seat_min_10','seat_min_11','seat_min_12','seat_min_13','seat_min_14',
                    'seat_min_15','seat_min_16','seat_min_17','seat_min_18','seat_min_19','seat_min_20','seat_min_21',
                    'seat_min_22','seat_min_23','seat_min_24','seat_min_25','seat_min_26','seat_min_27','seat_min_28',
                    'seat_max_8','seat_max_9','seat_max_10','seat_max_11','seat_max_12','seat_max_13','seat_max_14',
                    'seat_max_15','seat_max_16','seat_max_17','seat_max_18','seat_max_19','seat_max_20','seat_max_21',
                    'seat_max_22','seat_max_23','seat_max_24','seat_max_25','seat_max_26','seat_max_27','seat_max_28',
                    'seat_avg_8','seat_avg_9','seat_avg_10','seat_avg_11','seat_avg_12','seat_avg_13','seat_avg_14',
                    'seat_avg_15','seat_avg_16','seat_avg_17','seat_avg_18','seat_avg_19','seat_avg_20','seat_avg_21',
                    'seat_avg_22','seat_avg_23','seat_avg_24','seat_avg_25','seat_avg_26','seat_avg_27','seat_avg_28',
                    'seat_std_8','seat_std_9','seat_std_10','seat_std_11','seat_std_12','seat_std_13','seat_std_14',
                    'seat_std_15','seat_std_16','seat_std_17','seat_std_18','seat_std_19','seat_std_20','seat_std_21',
                    'seat_std_22','seat_std_23','seat_std_24','seat_std_25','seat_std_26','seat_std_27','seat_std_28',
                    'b_8','b_9','b_10','b_11','b_12','b_13','b_14','b_15','b_16','b_17','b_18','b_19','b_20','b_21','b_22','b_23','b_24','b_25','b_26','b_27','b_28',
                    'f_8','f_9','f_10','f_11','f_12','f_13','f_14','f_15','f_16','f_17','f_18','f_19','f_20','f_21','f_22','f_23','f_24','f_25','f_26','f_27','f_28',
                    'd_8','d_9','d_10','d_11','d_12','d_13','d_14','d_15','d_16','d_17','d_18','d_19','d_20','d_21','d_22','d_23','d_24','d_25','d_26','d_27','d_28',
                    's_8','s_9','s_10','s_11','s_12','s_13','s_14','s_15','s_16','s_17','s_18','s_19','s_20','s_21','s_22','s_23','s_24','s_25','s_26','s_27','s_28',
                    'sd_8','sd_9','sd_10','sd_11','sd_12','sd_13','sd_14','sd_15','sd_16','sd_17','sd_18','sd_19','sd_20','sd_21','sd_22','sd_23','sd_24','sd_25','sd_26','sd_27','sd_28']

    air.columns = rename
    
                    
    datetime_df = air[['DT']]
    ticket_df = air[['ticket_8', 'ticket_9', 'ticket_10','ticket_11','ticket_12','ticket_13','ticket_14',
                    'ticket_15','ticket_16', 'ticket_17','ticket_18','ticket_19','ticket_20','ticket_21',
                    'ticket_22', 'ticket_23', 'ticket_24','ticket_25','ticket_26','ticket_27','ticket_28']]
    faremin_df = air[['fare_min_8','fare_min_9','fare_min_10','fare_min_11','fare_min_12','fare_min_13','fare_min_14',
                    'fare_min_15','fare_min_16','fare_min_17','fare_min_18','fare_min_19','fare_min_20','fare_min_21',
                    'fare_min_22','fare_min_23','fare_min_24','fare_min_25','fare_min_26','fare_min_27','fare_min_28']]
    faremax_df = air[['fare_max_8','fare_max_9','fare_max_10','fare_max_11','fare_max_12','fare_max_13','fare_max_14',
                    'fare_max_15','fare_max_16','fare_max_17','fare_max_18','fare_max_19','fare_max_20','fare_max_21',
                    'fare_max_22','fare_max_23','fare_max_24','fare_max_25','fare_max_26','fare_max_27','fare_max_28']]
    fareavg_df = air[['fare_avg_8','fare_avg_9','fare_avg_10','fare_avg_11','fare_avg_12','fare_avg_13','fare_avg_14',
                    'fare_avg_15','fare_avg_16','fare_avg_17','fare_avg_18','fare_avg_19','fare_avg_20','fare_avg_21',
                    'fare_avg_22','fare_avg_23','fare_avg_24','fare_avg_25','fare_avg_26','fare_avg_27','fare_avg_28']]
    farestd_df = air[['fare_std_8','fare_std_9','fare_std_10','fare_std_11','fare_std_12','fare_std_13','fare_std_14',
                    'fare_std_15','fare_std_16','fare_std_17','fare_std_18','fare_std_19','fare_std_20','fare_std_21',
                    'fare_std_22','fare_std_23','fare_std_24','fare_std_25','fare_std_26','fare_std_27','fare_std_28']]
    seatmin_df = air[['seat_min_8','seat_min_9','seat_min_10','seat_min_11','seat_min_12','seat_min_13','seat_min_14',
                    'seat_min_15','seat_min_16','seat_min_17','seat_min_18','seat_min_19','seat_min_20','seat_min_21',
                    'seat_min_22','seat_min_23','seat_min_24','seat_min_25','seat_min_26','seat_min_27','seat_min_28']]
    seatmax_df = air[['seat_max_8','seat_max_9','seat_max_10','seat_max_11','seat_max_12','seat_max_13','seat_max_14',
                    'seat_max_15','seat_max_16','seat_max_17','seat_max_18','seat_max_19','seat_max_20','seat_max_21',
                    'seat_max_22','seat_max_23','seat_max_24','seat_max_25','seat_max_26','seat_max_27','seat_max_28']]
    seatavg_df = air[['seat_avg_8','seat_avg_9','seat_avg_10','seat_avg_11','seat_avg_12','seat_avg_13','seat_avg_14',
                    'seat_avg_15','seat_avg_16','seat_avg_17','seat_avg_18','seat_avg_19','seat_avg_20','seat_avg_21',
                    'seat_avg_22','seat_avg_23','seat_avg_24','seat_avg_25','seat_avg_26','seat_avg_27','seat_avg_28']]
    seatstd_df = air[['seat_std_8','seat_std_9','seat_std_10','seat_std_11','seat_std_12','seat_std_13','seat_std_14',
                    'seat_std_15','seat_std_16','seat_std_17','seat_std_18','seat_std_19','seat_std_20','seat_std_21',
                    'seat_std_22','seat_std_23','seat_std_24','seat_std_25','seat_std_26','seat_std_27','seat_std_28']]
    business_df = air[['b_8','b_9','b_10','b_11','b_12','b_13','b_14','b_15','b_16','b_17','b_18','b_19','b_20','b_21','b_22','b_23','b_24','b_25','b_26','b_27','b_28']]
    first_df =    air[['f_8','f_9','f_10','f_11','f_12','f_13','f_14','f_15','f_16','f_17','f_18','f_19','f_20','f_21','f_22','f_23','f_24','f_25','f_26','f_27','f_28']]
    discount_df = air[['d_8','d_9','d_10','d_11','d_12','d_13','d_14','d_15','d_16','d_17','d_18','d_19','d_20','d_21','d_22','d_23','d_24','d_25','d_26','d_27','d_28']]
    special_df =  air[['s_8','s_9','s_10','s_11','s_12','s_13','s_14','s_15','s_16','s_17','s_18','s_19','s_20','s_21','s_22','s_23','s_24','s_25','s_26','s_27','s_28']]
    speciald_df = air[['sd_8','sd_9','sd_10','sd_11','sd_12','sd_13','sd_14','sd_15','sd_16','sd_17','sd_18','sd_19','sd_20','sd_21','sd_22','sd_23','sd_24','sd_25','sd_26','sd_27','sd_28']]


    ticket_df = ticket_df.apply(lambda row: row.fillna(row.mean()), axis=1)
    faremin_df = faremin_df.apply(lambda row: row.fillna(row.mean()), axis=1)
    faremax_df = faremax_df.apply(lambda row: row.fillna(row.mean()), axis=1)
    fareavg_df = fareavg_df.apply(lambda row: row.fillna(row.mean()), axis=1)
    farestd_df = farestd_df.apply(lambda row: row.fillna(row.mean()), axis=1)
    seatmin_df = seatmin_df.apply(lambda row: row.fillna(row.mean()), axis=1)
    seatmax_df = seatmax_df.apply(lambda row: row.fillna(row.mean()), axis=1)
    seatavg_df = seatavg_df.apply(lambda row: row.fillna(row.mean()), axis=1)
    seatstd_df = seatstd_df.apply(lambda row: row.fillna(row.mean()), axis=1)
    business_df = business_df.apply(lambda row: row.fillna(row.mean()), axis=1)
    fitst_df = first_df.apply(lambda row: row.fillna(row.mean()), axis=1)
    discount_df = discount_df.apply(lambda row: row.fillna(row.mean()), axis=1)
    special_df = special_df.apply(lambda row: row.fillna(row.mean()), axis=1)
    speciald_df = speciald_df.apply(lambda row: row.fillna(row.mean()), axis=1)

    air2 = pd.concat([datetime_df,ticket_df, faremin_df, faremax_df, fareavg_df, farestd_df, seatmin_df, seatmax_df, seatavg_df, seatstd_df, business_df, fitst_df, discount_df, special_df, speciald_df], axis = 1)
    
    return air2

#### 데이터 통합 및 피쳐 엔지니어링

In [10]:
## Data Merge
def data_merge(psg_df, wth_h_df, air_df, rc_df, tw_df, start_date):
    merged = pd.merge(psg_df, wth_h_df, on="DT", how="left")
    merged = pd.merge(merged, air_df, on='DT', how='left')
    merged = pd.merge(merged, rc_df, on='DT', how='left')
    merged = pd.merge(merged, tw_df, on='DT', how='left')
    
    
    # columns rename
    merged.columns = ['datetime' , 'count', 'holiday', 'dayofweek',
                      'long_h','b_long_h','inter_h','b_inter_h','first_m','b_first_m',
                    'ticket_8', 'ticket_9', 'ticket_10','ticket_11','ticket_12','ticket_13','ticket_14',
                    'ticket_15','ticket_16', 'ticket_17','ticket_18','ticket_19','ticket_20','ticket_21',
                    'ticket_22', 'ticket_23', 'ticket_24','ticket_25','ticket_26','ticket_27','ticket_28',
                    'fare_min_8','fare_min_9','fare_min_10','fare_min_11','fare_min_12','fare_min_13','fare_min_14',
                    'fare_min_15','fare_min_16','fare_min_17','fare_min_18','fare_min_19','fare_min_20','fare_min_21',
                    'fare_min_22','fare_min_23','fare_min_24','fare_min_25','fare_min_26','fare_min_27','fare_min_28',
                    'fare_max_8','fare_max_9','fare_max_10','fare_max_11','fare_max_12','fare_max_13','fare_max_14',
                    'fare_max_15','fare_max_16','fare_max_17','fare_max_18','fare_max_19','fare_max_20','fare_max_21',
                    'fare_max_22','fare_max_23','fare_max_24','fare_max_25','fare_max_26','fare_max_27','fare_max_28',
                    'fare_avg_8','fare_avg_9','fare_avg_10','fare_avg_11','fare_avg_12','fare_avg_13','fare_avg_14',
                    'fare_avg_15','fare_avg_16','fare_avg_17','fare_avg_18','fare_avg_19','fare_avg_20','fare_avg_21',
                    'fare_avg_22','fare_avg_23','fare_avg_24','fare_avg_25','fare_avg_26','fare_avg_27','fare_avg_28',
                    'fare_std_8','fare_std_9','fare_std_10','fare_std_11','fare_std_12','fare_std_13','fare_std_14',
                    'fare_std_15','fare_std_16','fare_std_17','fare_std_18','fare_std_19','fare_std_20','fare_std_21',
                    'fare_std_22','fare_std_23','fare_std_24','fare_std_25','fare_std_26','fare_std_27','fare_std_28',
                    'seat_min_8','seat_min_9','seat_min_10','seat_min_11','seat_min_12','seat_min_13','seat_min_14',
                    'seat_min_15','seat_min_16','seat_min_17','seat_min_18','seat_min_19','seat_min_20','seat_min_21',
                    'seat_min_22','seat_min_23','seat_min_24','seat_min_25','seat_min_26','seat_min_27','seat_min_28',
                    'seat_max_8','seat_max_9','seat_max_10','seat_max_11','seat_max_12','seat_max_13','seat_max_14',
                    'seat_max_15','seat_max_16','seat_max_17','seat_max_18','seat_max_19','seat_max_20','seat_max_21',
                    'seat_max_22','seat_max_23','seat_max_24','seat_max_25','seat_max_26','seat_max_27','seat_max_28',
                    'seat_avg_8','seat_avg_9','seat_avg_10','seat_avg_11','seat_avg_12','seat_avg_13','seat_avg_14',
                    'seat_avg_15','seat_avg_16','seat_avg_17','seat_avg_18','seat_avg_19','seat_avg_20','seat_avg_21',
                    'seat_avg_22','seat_avg_23','seat_avg_24','seat_avg_25','seat_avg_26','seat_avg_27','seat_avg_28',
                    'seat_std_8','seat_std_9','seat_std_10','seat_std_11','seat_std_12','seat_std_13','seat_std_14',
                    'seat_std_15','seat_std_16','seat_std_17','seat_std_18','seat_std_19','seat_std_20','seat_std_21',
                    'seat_std_22','seat_std_23','seat_std_24','seat_std_25','seat_std_26','seat_std_27','seat_std_28',
                    'b_8','b_9','b_10','b_11','b_12','b_13','b_14','b_15','b_16','b_17','b_18','b_19','b_20','b_21','b_22','b_23','b_24','b_25','b_26','b_27','b_28',
                    'f_8','f_9','f_10','f_11','f_12','f_13','f_14','f_15','f_16','f_17','f_18','f_19','f_20','f_21','f_22','f_23','f_24','f_25','f_26','f_27','f_28',
                    'd_8','d_9','d_10','d_11','d_12','d_13','d_14','d_15','d_16','d_17','d_18','d_19','d_20','d_21','d_22','d_23','d_24','d_25','d_26','d_27','d_28',
                    's_8','s_9','s_10','s_11','s_12','s_13','s_14','s_15','s_16','s_17','s_18','s_19','s_20','s_21','s_22','s_23','s_24','s_25','s_26','s_27','s_28',
                    'sd_8','sd_9','sd_10','sd_11','sd_12','sd_13','sd_14','sd_15','sd_16','sd_17','sd_18','sd_19','sd_20','sd_21','sd_22','sd_23','sd_24','sd_25','sd_26','sd_27','sd_28',
                      'lt_8',   'lt_9',   'lt_10',  'lt_11',  'lt_12',  'lt_13', 'lt_14',
                      'lt_15',   'lt_16',   'lt_17',  'lt_18',  'lt_19',  'lt_20', 'lt_21',
                      'lt_22',   'lt_23',   'lt_24',   'lt_25',   'lt_26',   'lt_27' , 'lt_28',   
                      'f330_8',   'f330_9',   'f330_10',  'f330_11',  'f330_12',  'f330_13', 'f330_14',
                      'f330_15',  'f330_16',  'f330_17',  'f330_18',  'f330_19',  'f330_20', 'f330_21',
                      'f330_22',  'f330_23',  'f330_24',  'f330_25',  'f330_26',  'f330_27' ,'f330_28', 
                      'f737_8',   'f737_9',   'f737_10',  'f737_11',  'f737_12',  'f737_13', 'f737_14',
                      'f737_15',  'f737_16',  'f737_17',  'f737_18',  'f737_19',  'f737_20', 'f737_21',
                      'f737_22',   'f737_23',   'f737_24',   'f737_25',   'f737_26',   'f737_27' , 'f737_28',   
                      'group_8',   'group_9',   'group_10',  'group_11',  'group_12',  'group_13', 'group_14',
                      'group_15',   'group_16',   'group_17',  'group_18',  'group_19',  'group_20', 'group_21',
                      'group_22',   'group_23',   'group_24',   'group_25',   'group_26',   'group_27' , 'group_28', 
                      'total_8',   'total_9',   'total_10',  'total_11',  'total_12',  'total_13', 'total_14',
                      'total_15',   'total_16',   'total_17',  'total_18',  'total_19',  'total_20', 'total_21',
                      'total_22',   'total_23',   'total_24',   'total_25',   'total_26',   'total_27' , 'total_28', ]

    
    # for visitor count lag
    merged_count=merged[['datetime','count']].sort_values(by='datetime')
    merged = merged.loc[merged.datetime >= start_date,:].sort_values(by='datetime')
    return merged, merged_count



## Feature Engineering
def feature_engineering(raw):
    if 'datetime' in raw.columns:
        raw['DateTime'] = pd.to_datetime(raw['datetime'])
    if raw.index.dtype == 'int64':
        raw.set_index('DateTime', inplace=True)
    
    # add time series data 
    #raw['month'] = raw.datetime.dt.month
    
    # dummy 변수(카테고리 변수) => holiday(0,1) / dayofweek(0,1,2,3,4,5,6) / long_h(0,1) / b_long_h(0,1) / inter_h(0,1) / b_inter_h(0,1) 
    #                         /first_m(0,1)/ b_first_m(0,1) / year(2019,2020,2021,2022) / month(1~12) / quarter(1~4)

    #raw = pd.concat([raw, pd.get_dummies(raw['holiday'],   prefix='holiday'   + '_dummy')], axis=1)
    raw = pd.concat([raw, pd.get_dummies(raw['dayofweek'], prefix='dayofweek' + '_dummy')], axis=1)
    #raw = pd.concat([raw, pd.get_dummies(raw['long_h'],    prefix='long_h'    + '_dummy')], axis=1)
    #raw = pd.concat([raw, pd.get_dummies(raw['b_long_h'],  prefix='b_long_h'  + '_dummy')], axis=1)
    #raw = pd.concat([raw, pd.get_dummies(raw['inter_h'],   prefix='inter_h'   + '_dummy')], axis=1)
    #raw = pd.concat([raw, pd.get_dummies(raw['b_inter_h'], prefix='b_inter_h' + '_dummy')], axis=1)
    #raw = pd.concat([raw, pd.get_dummies(raw['first_m'],   prefix='first_m'   + '_dummy')], axis=1)
    #raw = pd.concat([raw, pd.get_dummies(raw['b_first_m'], prefix='b_first_m' + '_dummy')], axis=1)
    #raw = pd.concat([raw, pd.get_dummies(raw['month'],     prefix='month'     + '_dummy')], axis=1)

    raw = raw.drop(columns=['dayofweek'])
    
    raw_fe = raw.copy() 
    return raw_fe


# Count lagged values of X_test
def feature_engineering_lag_modified(Y, X, target):
    X_lm = X.copy()
    i = 0
    for col in target:
        X_lm[col] = Y.shift(i+14).values
        X_lm[col].fillna(method='bfill', inplace=True)
        i = i + 1
    return X_lm

# Data Split for Y & X
def datasplit_X_Y(data, Y_colname, X_colname):
    X_colname = [x for x in data.columns if x not in Y_colname + X_remove]
    Y_data = data[Y_colname]
    X_data = data[X_colname]
    print('X:', X_data.shape, 'Y:', Y_data.shape)
    return X_data, Y_data

# Data Split for time series to train & test
def datasplit_ts(raw, train_ratio):
    size = int(len(raw) * train_ratio)
    raw_train, raw_val = raw[0:size].copy(deep=True), raw[size:len(raw)].copy(deep=True)
    print('Train_size:', raw_train.shape, 'Validation_size:', raw_val.shape)
    return raw_train, raw_val

# def datasplit_ts(raw, criteria):
#     raw_train = raw.loc[merged.index < criteria, :]
#     raw_validation  = raw.loc[merged.index >= criteria, :]
#     print('Train_size:', raw_train.shape, 'Validation_size:', raw_validation.shape)
#     return raw_train, raw_validation

#### 기간 설정 및 데이터 불러오기

In [11]:
today = date.today()
yesterday = date.today() - timedelta(1)
yesterday=yesterday.strftime('%Y-%m-%d')

In [12]:
psg, air, wth, rc, tw = data_load(query_AP, query_AT, query_HD, query_RC, query_TW, '2022-01-01', yesterday)

In [13]:
air = air.pivot(index='DT', columns='LEAD_TM', values=['TICKET', 'FARE_MIN', 'FARE_MAX', 'FARE_AVG', 'FARE_STD', 'SEAT_MIN', 'SEAT_MAX', 'SEAT_AVG', 'SEAT_STD', 'B', 'F', 'D', 'S', 'SD'])
air.columns = air.columns.to_flat_index()
air=air.reset_index()
air2=air_dataprocessing(air)

## function 실행

In [14]:
## Holiday Data Preprocessing
wth_h = holiday_data_pre(wth)

In [15]:
rc = rentacar_processing(rc)
tw = tway_processing(tw)

In [16]:
## Data Merge
merged, merged_count = data_merge(psg, wth_h, air2, rc, tw, '2022-02-02')

In [17]:
# Feature Engineering
merged_fe  = feature_engineering(merged)

# Modify count lagged values of X_train
target_l = ['count_lag14','count_lag15', 'count_lag16','count_lag17', 'count_lag18','count_lag19','count_lag20']
count_lag = feature_engineering_lag_modified(pd.DataFrame(merged_count['count']), pd.DataFrame(merged_count['datetime']), target_l)
merged_fe=pd.merge(merged_fe, count_lag, on='datetime', how='left')

# Data Split to X and Y
Y_colname = ['count']
X_remove = ['DateTime']
X_colname = [x for x in merged_fe.columns if x not in Y_colname + X_remove]
X, Y = datasplit_X_Y(merged_fe, Y_colname, X_colname)

# Data Split for time series to train & test
X_train, X_validation = datasplit_ts(X, 0.90)
Y_train, Y_validation = datasplit_ts(Y, 0.90)

X: (204, 421) Y: (204, 1)
Train_size: (183, 421) Validation_size: (21, 421)
Train_size: (183, 1) Validation_size: (21, 1)


In [18]:
Y

,count
0,44620
1,41036
2,36365
3,34202
4,40057
...,...
199,40362
200,42797
201,43548
202,39563


In [19]:
trainds= pd.concat([X_train, Y_train], axis=1)
valds = pd.concat([X_validation, Y_validation], axis=1)

In [20]:
trainds['data_split']='TRAIN'
valds['data_split'] ='VALIDATE'

In [21]:
trainds.fillna(method = 'ffill',inplace=True) 

valds.fillna(method = 'ffill', inplace=True) 

In [22]:
table_id = "charged-genre-350106.demand_forecasting.airport_passenger_tr_nm_ns_14_lt21_tr_fm"

job_config = bigquery.job.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

bqclient.load_table_from_dataframe(trainds, table_id, job_config=job_config)

LoadJob<project=charged-genre-350106, location=asia-northeast3, id=6ce40d9d-ec19-4071-a58e-56ab6cc4546f>

In [23]:
table_id = "charged-genre-350106.demand_forecasting.airport_passenger_ts_nm_ns_14_lt21_tr_fm"

job_config = bigquery.job.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

bqclient.load_table_from_dataframe(valds, table_id, job_config=job_config)

LoadJob<project=charged-genre-350106, location=asia-northeast3, id=406fadac-866c-48c2-876b-0050e08069ff>